# Проект: Сопоставление гео названий с унифицированными именами geonames

## Заказчик

Карьерный центр

## Описание проекта

**Цель:**

-Сопоставление произвольных гео названий с унифицированными именами geonames для внутреннего использования Карьерным центром

**Используемый стек технологий:**

- Класс для использования на backend: Pyhton
- База данных: Postgresql
- Модели: 
    - Count Vectorizer и 
    - Предъобученная модель Sbert - Labse (https://sbert.net/docs/pretrained_models.html)
    - Предъобученная модель Sbert - Labse-geonames (https://huggingface.co/dima-does-code/LaBSE-geonames-15K-MBML-5e-v1)
    - Поиск по косинусному сравнению

**Может потребоваться для запуска**

- pip install translate
- pip install SQLAlchemy
- pip install -U sentence-transformers

**Данные geonames:**

В проекте были использованы файлы:
- allCountries.txt - файл таблицы geoname. Содержит все данные сервиса geoname
- alternateNamesV2.txt - файл с альтернативными названиям для гео объектов
- admin1CodesASCII.txt - файл с кодами регионального деления
- countryInfo.txt - файл с информацией по странам

В проекте были использованы следующие фильтры:
- страны: ['RU', 'AM', 'AZ', 'BY', 'GE', 'KG', 'KZ', 'MD', 'TJ', 'UA', 'UZ']
- языки: ['ru', 'az', 'en', 'tr', 'uz', 'abbr', 'iata', 'icao', 'faac']
- класс объектов: ['P'] - населенные пункты
- кол-во населения: от 5000 чел.

## Инициализация класса

In [ ]:
from citysearch import citysearch

path = 'C:/Users/sexypc/ds'

DATABASE = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': '**********',
    'host': 'localhost',
    'port': 5432,
    'database': 'geonames',
    'query': {}
}

citysearch = citysearch(path, DATABASE)

## Инициализация базы данных

Создается чистая структура и закачиваются данные по отфильтрованным данным

In [ ]:
citysearch.db_load_data(path, DATABASE)

## CountVectrizer + Cosinesimlarty

Векторы хранятся и извлекаются из базы данных. Для Count Vectorizer этот вариант приемлемый т.к. векторы имеют относительно маленький размер.

### Расчет embeddings

Функция calc_count_vectorizer рассчитывает Embeddings для всех значений names и alternames из данных geonames. Далее сохраняет полученные векторы в таблицу `CountVectorizer` в базе данных.

In [ ]:
%%time

citysearch.calc_count_vectorizer()

### Поиск города

Извлекаются Embeddings из базы данных и происходит поиск.

In [40]:
%%time

search = 'Maskva'
citysearch.get_CV_similarity(search, translate=True)

CPU times: total: 688 ms
Wall time: 2.09 s


[{'geoname_id': 470676,
  'name': 'Vyaz’ma',
  'country': 'Russia',
  'region': 'Smolensk Oblast',
  'population': 55500,
  'cosine_similarity': 1.0},
 {'geoname_id': 702320,
  'name': 'Makiivka',
  'country': 'Ukraine',
  'region': 'Donetsk',
  'population': 338968,
  'cosine_similarity': 0.943}]

In [42]:
%%time

search = 'Моссква'
citysearch.get_CV_similarity(search, translate=True)

CPU times: total: 672 ms
Wall time: 1.93 s


[{'geoname_id': 524901,
  'name': 'Moscow',
  'country': 'Russia',
  'region': 'Moscow',
  'population': 10381222,
  'cosine_similarity': 1.0},
 {'geoname_id': 1498389,
  'name': 'Moshkovo',
  'country': 'Russia',
  'region': 'Novosibirsk Oblast',
  'population': 10541,
  'cosine_similarity': 0.912},
 {'geoname_id': 610612,
  'name': 'Shevchenko',
  'country': 'Kazakhstan',
  'region': 'Mangghystaū',
  'population': 147443,
  'cosine_similarity': 0.873},
 {'geoname_id': 1504769,
  'name': 'Kamyshlov',
  'country': 'Russia',
  'region': 'Sverdlovsk Oblast',
  'population': 28006,
  'cosine_similarity': 0.857}]

In [31]:
%%time

search = 'Санкт-петербург'
citysearch.get_CV_similarity(search, translate=True)

CPU times: total: 375 ms
Wall time: 1.89 s


[{'geoname_id': 498817,
  'name': 'Saint Petersburg',
  'country': 'Russia',
  'region': 'St.-Petersburg',
  'population': 5351935,
  'cosine_similarity': 1.0}]

In [33]:
%%time

search = 'Vlodivastok'
citysearch.get_CV_similarity(search, translate=True)

CPU times: total: 500 ms
Wall time: 1.63 s


[{'geoname_id': 2013348,
  'name': 'Vladivostok',
  'country': 'Russia',
  'region': 'Primorye',
  'population': 604901,
  'cosine_similarity': 1.0},
 {'geoname_id': 694423,
  'name': 'Sevastopol',
  'country': 'Ukraine',
  'region': 'Sevastopol City',
  'population': 547820,
  'cosine_similarity': 0.93},
 {'geoname_id': 692087,
  'name': 'Svitlovodsk',
  'country': 'Ukraine',
  'region': 'Kirovohrad',
  'population': 43130,
  'cosine_similarity': 0.926},
 {'geoname_id': 512023,
  'name': 'Pavlovskiy Posad',
  'country': 'Russia',
  'region': 'Moscow Oblast',
  'population': 60051,
  'cosine_similarity': 0.897}]

In [44]:
%%time

search = 'Kirov'
citysearch.get_CV_similarity(search, translate=True)

CPU times: total: 734 ms
Wall time: 2.25 s


[{'geoname_id': 548408,
  'name': 'Kirov',
  'country': 'Russia',
  'region': 'Kirov Oblast',
  'population': 507155,
  'cosine_similarity': 1.0},
 {'geoname_id': 548410,
  'name': 'Kirov',
  'country': 'Russia',
  'region': 'Kaluga Oblast',
  'population': 39319,
  'cosine_similarity': 1.0},
 {'geoname_id': 548391,
  'name': 'Kirovsk',
  'country': 'Russia',
  'region': 'Murmansk',
  'population': 29605,
  'cosine_similarity': 1.0},
 {'geoname_id': 705809,
  'name': 'Holubivka',
  'country': 'Ukraine',
  'region': 'Luhansk',
  'population': 26654,
  'cosine_similarity': 1.0}]

**Вывод** 

Поиск по ngram будет работать с относительно схожими словами. Может справляться с опечатками. Требуется перевод слова на английский язык. Была использована библиотека для перевода translate. Если библиотека не может перевести, она транслитирирует в латинский алфавит.

## Cosine similarity SBERT distiluse-base-multilingual-cased-v2

Данные методы работают аналогичным образом

### Расчет embeddings

In [ ]:
%%time

citysearch.calc_sbertv2()

### Поиск города

In [ ]:
%%time

string = 'RUMOW'
citysearch.get_sbertv2_similarity(string)

**Вывод**

Хранить в базе эмбединги плохая идея. Очень долгое чтение из базы данных. Необходимо сохранять после первого запроса в переменную класса.

**Рекомендации**

Не использовать

## Cosine similarity SBERT dima-does-code/LaBSE-geonames-15K-MBML-5e-v1

Аналогичные проблемы со скоростью чтения embeddings из базы данных

### Расчет embeddings

In [ ]:
%%time

citysearch.calc_sbert_Labse()

### Поиск города

In [ ]:
%%time

string = 'RUMOW'
citysearch.get_sbert_Labse_similarity(string)

**Рекомендации**

Не использовать

## Cosine similarity SBERT dima-does-code/LaBSE-geonames-15K-MBML-5e-v1
## Версия 2

За основу была взята модель SBERT LaBSE и дообучена на парах данных: альтернативное название - название. Модель обучалась на 5 эпохах. Данна модель мультиязычная. Поддерживает ввод и поиск названий городов независимо от языка. 

### Расчет embeddings

In [ ]:
%%time

citysearch.calc_sbert_Labse2()

### Поиск города

In [45]:
%%time

string = 'Масква'
citysearch.get_sbert_Labse_similarity2(string)

CPU times: total: 172 ms
Wall time: 235 ms


[{'geoname_id': 524901,
  'name': 'Moscow',
  'country': 'Russia',
  'region': 'Moscow',
  'population': 10381222,
  'similarity': 0.9391975402832031},
 {'geoname_id': 1498389,
  'name': 'Moshkovo',
  'country': 'Russia',
  'region': 'Novosibirsk Oblast',
  'population': 10541,
  'similarity': 0.7764918208122253}]

In [46]:
%%time

string = 'Моссква'
citysearch.get_sbert_Labse_similarity2(string)

CPU times: total: 188 ms
Wall time: 87.1 ms


[{'geoname_id': 524901,
  'name': 'Moscow',
  'country': 'Russia',
  'region': 'Moscow',
  'population': 10381222,
  'similarity': 0.9065966010093689},
 {'geoname_id': 1498389,
  'name': 'Moshkovo',
  'country': 'Russia',
  'region': 'Novosibirsk Oblast',
  'population': 10541,
  'similarity': 0.8068616390228271}]

In [47]:
%%time

string = 'СПб'
citysearch.get_sbert_Labse_similarity2(string)

CPU times: total: 172 ms
Wall time: 91.3 ms


[{'geoname_id': 498817,
  'name': 'Saint Petersburg',
  'country': 'Russia',
  'region': 'St.-Petersburg',
  'population': 5351935,
  'similarity': 0.9999999403953552}]

In [48]:
%%time

string = 'Мск'
citysearch.get_sbert_Labse_similarity2(string)

CPU times: total: 219 ms
Wall time: 102 ms


[{'geoname_id': 1498129,
  'name': 'Myski',
  'country': 'Russia',
  'region': 'Kuzbass',
  'population': 44082,
  'similarity': 0.8076306581497192},
 {'geoname_id': 524901,
  'name': 'Moscow',
  'country': 'Russia',
  'region': 'Moscow',
  'population': 10381222,
  'similarity': 0.7616719603538513}]

In [49]:
%%time

string = 'МСК'
citysearch.get_sbert_Labse_similarity2(string)

CPU times: total: 203 ms
Wall time: 98.3 ms


[{'geoname_id': 625144,
  'name': 'Minsk',
  'country': 'Belarus',
  'region': 'Minsk City',
  'population': 1742124,
  'similarity': 0.7677884697914124},
 {'geoname_id': 532096,
  'name': 'Makhachkala',
  'country': 'Russia',
  'region': 'Dagestan',
  'population': 596356,
  'similarity': 0.7533031105995178}]

In [52]:
%%time

string = 'Навосибирсг'
citysearch.get_sbert_Labse_similarity2(string)

CPU times: total: 188 ms
Wall time: 89.6 ms


[{'geoname_id': 1496747,
  'name': 'Novosibirsk',
  'country': 'Russia',
  'region': 'Novosibirsk Oblast',
  'population': 1612833,
  'similarity': 0.7570608854293823}]

In [53]:
%%time

string = 'Влодивасток'
citysearch.get_sbert_Labse_similarity2(string)

CPU times: total: 203 ms
Wall time: 84.7 ms


[{'geoname_id': 2013348,
  'name': 'Vladivostok',
  'country': 'Russia',
  'region': 'Primorye',
  'population': 604901,
  'similarity': 0.8974615931510925},
 {'geoname_id': 473249,
  'name': 'Vladikavkaz',
  'country': 'Russia',
  'region': 'North Ossetia–Alania',
  'population': 306258,
  'similarity': 0.7779488563537598}]

In [55]:
%%time

string = 'Moskau'
citysearch.get_sbert_Labse_similarity2(string)

CPU times: total: 203 ms
Wall time: 86.5 ms


[{'geoname_id': 524901,
  'name': 'Moscow',
  'country': 'Russia',
  'region': 'Moscow',
  'population': 10381222,
  'similarity': 0.9999999403953552},
 {'geoname_id': 1498389,
  'name': 'Moshkovo',
  'country': 'Russia',
  'region': 'Novosibirsk Oblast',
  'population': 10541,
  'similarity': 0.7683038711547852}]

In [58]:
%%time

string = 'ქუთაისი'
citysearch.get_sbert_Labse_similarity2(string)

CPU times: total: 266 ms
Wall time: 95.5 ms


[{'geoname_id': 613607,
  'name': 'Kutaisi',
  'country': 'Georgia',
  'region': 'Imereti',
  'population': 135201,
  'similarity': 0.8971949815750122}]

In [59]:
%%time

string = 'Kürdəmir'
citysearch.get_sbert_Labse_similarity2(string)

CPU times: total: 156 ms
Wall time: 86.5 ms


[{'geoname_id': 585763,
  'name': 'Kyurdarmir',
  'country': 'Azerbaijan',
  'region': 'Kürdǝmir',
  'population': 19088,
  'similarity': 1.0000003576278687}]

In [60]:
%%time

string = 'G‘ijduvon'
citysearch.get_sbert_Labse_similarity2(string)

CPU times: total: 141 ms
Wall time: 88.7 ms


[{'geoname_id': 1513983,
  'name': 'G’ijduvon Shahri',
  'country': 'Uzbekistan',
  'region': 'Bukhara',
  'population': 41070,
  'similarity': 0.9649820923805237}]

In [63]:
%%time

string = 'Krasnyy Oktyabr'
citysearch.get_sbert_Labse_similarity2(string)

CPU times: total: 172 ms
Wall time: 87 ms


[{'geoname_id': 541549,
  'name': 'Krasnyy Oktyabr’',
  'country': 'Russia',
  'region': 'Vladimir Oblast',
  'population': 9999,
  'similarity': 0.9913536310195923},
 {'geoname_id': 608679,
  'name': 'Kandyagash',
  'country': 'Kazakhstan',
  'region': 'Aqtöbe',
  'population': 28196,
  'similarity': 0.9031741619110107},
 {'geoname_id': 515879,
  'name': 'Oktyabrsky',
  'country': 'Russia',
  'region': 'Bashkortostan Republic',
  'population': 108200,
  'similarity': 0.8739874362945557},
 {'geoname_id': 515873,
  'name': 'Oktyabr’skiy',
  'country': 'Russia',
  'region': 'Moscow Oblast',
  'population': 11000,
  'similarity': 0.8739874362945557},
 {'geoname_id': 515921,
  'name': 'Takhtamukay',
  'country': 'Russia',
  'region': 'Adygeya Republic',
  'population': 5120,
  'similarity': 0.8739874362945557}]

In [68]:
%%time

string = 'Қарағанды'
citysearch.get_sbert_Labse_similarity2(string)

CPU times: total: 203 ms
Wall time: 73.1 ms


[{'geoname_id': 609655,
  'name': 'Karagandy',
  'country': 'Kazakhstan',
  'region': 'Karaganda',
  'population': 497777,
  'similarity': 0.8726395964622498},
 {'geoname_id': 2013894,
  'name': 'Ust’-Ordynskiy',
  'country': 'Russia',
  'region': 'Irkutsk Oblast',
  'population': 14538,
  'similarity': 0.7625464200973511}]

**Todo** 

Данная модель сохраняет эмбеддинги в файл с помощью pickle. 

Первый поиск названия долгий. Необходимо оптимизировать сохранение эмбеддингов в файл. (Добиться, чтобы хранились в правильном виде, без необходимости трансформации типов после чтения их из файла.)

### Тест

# TODO

    [] Добавить передачу фильтров в виде словаря
    [] Добавить проверки передаваемых параметров
    [x] Оформить проект на Github
    [] Провести тесты на тестовых данных
    [] Посмотреть пропущенные вебинары
    [] Обучить свою собственную модель на основе sbert
    [] Оформить страницу на HuggingFace
    [] Сделать рефакторинг класса и оформить в модуль
    [] Дополню как вспомню